In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import pickle

import xarray as xr
import rioxarray

import rasterio
from rasterio.windows import Window, from_bounds
from rasterio.warp import Resampling
from rasterio.vrt import WarpedVRT
from rasterio import shutil as rio_shutil

/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/Sun-jooLee/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# paths
dirpath = '/Users/Sun-jooLee/Git_Projects/Myanmar_Survey_ML/data/geo'
# this is the reference grid - landscan (population)
refpath = f'{dirpath}/landscan/landscan-global-2017.tif'

In [3]:
# myanmar bounding box
min_lon = 92.3032344909 
min_lat = 9.93295990645 
max_lon = 101.180005324 
max_lat = 28.335945136 

### Reference grid vrt options

In [4]:
with rasterio.open(refpath) as src:
    # clip to myanmar
    win = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    win = Window(*[max(0, v) for v in win.flatten()])
    vrt_options = {
        "crs": src.crs.to_epsg(),
        "transform": src.window_transform(win),
        "width": int(win.width),
        "height": int(win.height),
    }
    
vrt_options.update({'resampling':Resampling.bilinear})
vrt_options

{'crs': 4326,
 'transform': Affine(0.0083333333333333, 0.0, 92.30323449089997,
        0.0, -0.0083333333333333, 28.335945136),
 'width': 1065,
 'height': 2208,
 'resampling': <Resampling.bilinear: 1>}

### Landscan (annual) - ref grid

In [5]:
# path
landscan_path = f'{dirpath}/landscan/landscan-global-2017.tif'
# open file
da = rioxarray.open_rasterio(landscan_path)
# clip to myanmar
lscn = da.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon, maxy=max_lat)
# fill non value 
lscn = lscn.where(lscn != lscn.attrs['_FillValue'], np.nan)
# some cleaning
lscn = lscn.drop('band').drop('spatial_ref')
lscn.name = 'landscan'
lscn = lscn.to_dataset()
lscn = (
    lscn.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['landscan']})
    .rename({'x':'lon', 'y':'lat'})
    .squeeze('band')
)

In [6]:
# check
lscn

<xarray.Dataset>
Dimensions:   (time: 1, source: 1, lon: 1066, lat: 2210)
Coordinates:
  * time      (time) object 2017-01-01
  * source    (source) object 'landscan'
  * lon       (lon) float64 92.3 92.31 92.32 92.33 ... 101.2 101.2 101.2 101.2
  * lat       (lat) float64 28.34 28.33 28.32 28.31 ... 9.954 9.946 9.937 9.929
Data variables:
    landscan  (time, source, lat, lon) float64 1.0 1.0 0.0 0.0 ... nan nan nan

In [7]:
# save
with open(f'{dirpath}/landscan/landscan2017_xarray.pickle', 'wb') as handle:
    pickle.dump(lscn, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Landcover (annual)

In [8]:
# path
landcover_path = f'netcdf:{dirpath}/landcover/C3S-LC-L4-LCCS-Map-300m-P1Y-2017-v2.1.1-002.nc:lccs_class'
# open transformed and clipped to myanmar
with rasterio.open(landcover_path) as src:
    with WarpedVRT(src, **vrt_options) as vrt:
        lcvr = rioxarray.open_rasterio(vrt)

In [9]:
# some cleaning - note, no non values in myanmar
lcvr = lcvr.drop('band').drop('spatial_ref')
lcvr.name = 'landcover'
lcvr = lcvr.to_dataset()
lcvr = (
    lcvr.expand_dims({'time':[datetime.date(2017,1,1)], 'source':['landcover']})
    .rename({'x':'lon', 'y':'lat'})
    .squeeze('band')
)

In [10]:
# check
lcvr

<xarray.Dataset>
Dimensions:    (time: 1, source: 1, lon: 1065, lat: 2208)
Coordinates:
  * time       (time) object 2017-01-01
  * source     (source) object 'landcover'
  * lon        (lon) float64 92.31 92.32 92.32 92.33 ... 101.1 101.2 101.2 101.2
  * lat        (lat) float64 28.33 28.32 28.32 28.31 ... 9.965 9.957 9.948 9.94
Data variables:
    landcover  (time, source, lat, lon) uint8 129 115 109 130 ... 210 210 210

In [11]:
# save
with open(f'{dirpath}/landcover/landcover2017_xarray.pickle', 'wb') as handle:
    pickle.dump(lcvr, handle, protocol=pickle.HIGHEST_PROTOCOL)

### VIIRS (monthly)

In [12]:
viirs_lst = [file for file in os.listdir(f'{dirpath}/viirs/2017/') if file!='.DS_Store']
viirs_xarrays = []

for file in viirs_lst:
    # path
    viirs_path = f'{dirpath}/viirs/2017/{file}'
    # open transformed and clipped to myanmar
    with rasterio.open(viirs_path) as src:
        with WarpedVRT(src, **vrt_options) as vrt:
            viirs = rioxarray.open_rasterio(vrt)
            
    # process one month dataset    
    yearmonth_tif = file.split('_')[-1]
    year = int(yearmonth_tif[:4])
    month = int(yearmonth_tif[4:6])
    
    # fill non value 
    viirs = viirs.where(viirs != viirs.attrs['_FillValue'], np.nan)
    # some cleaning
    viirs = viirs.drop('band').drop('spatial_ref')
    viirs.name = 'viirs'
    viirs = viirs.to_dataset()
    viirs = (
        viirs.expand_dims({'time':[datetime.date(year, month, 1)], 'source':['viirs']})
        .rename({'x':'lon', 'y':'lat'})
        .squeeze('band')
    )
    viirs_xarrays.append(viirs)

In [13]:
# check
print(len(viirs_xarrays))
viirs_all = xr.combine_by_coords(viirs_xarrays, combine_attrs='drop')
viirs_all

12


<xarray.Dataset>
Dimensions:  (time: 12, source: 1, lon: 1065, lat: 2208)
Coordinates:
  * time     (time) object 2017-01-01 2017-02-01 ... 2017-11-01 2017-12-01
  * source   (source) object 'viirs'
  * lon      (lon) float64 92.31 92.32 92.32 92.33 ... 101.1 101.2 101.2 101.2
  * lat      (lat) float64 28.33 28.32 28.32 28.31 ... 9.965 9.957 9.948 9.94
Data variables:
    viirs    (time, source, lat, lon) float32 0.1052 0.08475 ... 0.1907 0.1586

In [14]:
# save
with open(f'{dirpath}/viirs/2017/viirs2017_xarray.pickle', 'wb') as handle:
    pickle.dump(viirs_all, handle, protocol=pickle.HIGHEST_PROTOCOL)

### FLDAS (monthly)

In [15]:
# all_vars = ['Evap_tavg','LWdown_f_tavg','Lwnet_tavg','Psurf_f_tavg','Qair_f_tavg','Qg_tavg','Qh_tavg','Qle_tavg','Qs_tavg','Qsb_tavg','RadT_tavg','Rainf_f_tavg','SWE_inst','SWdown_f_tavg','SnowCover_inst','SnowDepth_inst','Snowf_tavg','Swnet_tavg','Tair_f_tavg','Wind_f_tavg','SoilMoi00_10cm_tavg','SoilMoi10_40cm_tavg','SoilMoi40_100cm_tavg','SoilMoi100_200cm_tavg','SoilTemp00_10cm_tavg','SoilTemp10_40cm_tavg','SoilTemp40_100cm_tavg','SoilTemp100_200cm_tavg']
fldas_vars = ['Evap_tavg','Rainf_f_tavg','Tair_f_tavg','SoilMoi00_10cm_tavg','SoilMoi10_40cm_tavg','SoilMoi40_100cm_tavg','SoilMoi100_200cm_tavg','Qair_f_tavg','Qg_tavg','Qh_tavg','Qs_tavg']

In [16]:
fldas_lst = [file for file in os.listdir(f'{dirpath}/fldas/2017/') if file!='.DS_Store']
fldas_xarrays = []

for file in fldas_lst:
    for var in fldas_vars:
        # path
        fldas_path = f'netcdf:{dirpath}/fldas/2017/{file}:{var}'
        # open transformed and clipped to myanmar
        with rasterio.open(fldas_path) as src:
            with WarpedVRT(src, **vrt_options) as vrt:
                fldas = rioxarray.open_rasterio(vrt)

        # process one month dataset    
        yearmonth = file.split('.')[1]
        year = int(yearmonth[1:5])
        month = int(yearmonth[5:7])
        
        # fill non value 
        fldas = fldas.where(fldas != fldas.attrs['_FillValue'], np.nan)
        # some cleaning
        fldas = fldas.drop('band').drop('spatial_ref')
        fldas.name = var
        fldas = fldas.to_dataset()
        fldas = (
            fldas.expand_dims({'time':[datetime.date(year, month, 1)], 'source':['fldas']})
            .rename({'x':'lon', 'y':'lat'})
            .squeeze('band')
        )
        fldas_xarrays.append(fldas)

In [17]:
# check
print(len(fldas_xarrays))
fldas_all = xr.combine_by_coords(fldas_xarrays, combine_attrs='drop')
fldas_all

132


<xarray.Dataset>
Dimensions:                (time: 12, source: 1, lon: 1065, lat: 2208)
Coordinates:
  * time                   (time) object 2017-01-01 2017-02-01 ... 2017-12-01
  * source                 (source) object 'fldas'
  * lon                    (lon) float64 92.31 92.32 92.32 ... 101.2 101.2 101.2
  * lat                    (lat) float64 28.33 28.32 28.32 ... 9.957 9.948 9.94
Data variables:
    Evap_tavg              (time, source, lat, lon) float32 1.634e-07 ... nan
    Qair_f_tavg            (time, source, lat, lon) float32 0.00178 ... nan
    Qg_tavg                (time, source, lat, lon) float32 -5.276 ... nan
    Qh_tavg                (time, source, lat, lon) float32 79.97 79.72 ... nan
    Qs_tavg                (time, source, lat, lon) float32 3.853e-09 ... nan
    Rainf_f_tavg           (time, source, lat, lon) float32 1.133e-09 ... nan
    SoilMoi00_10cm_tavg    (time, source, lat, lon) float32 0.1695 ... nan
    SoilMoi100_200cm_tavg  (time, source, lat, lon) float32 0.1437 ... nan
    SoilMoi10_40cm_tavg    (time, source, lat, lon) float32 0.2384 ... nan
    SoilMoi40_100cm_tavg   (time, source, lat, lon) float32 0.1943 0.198 ... nan
    Tair_f_tavg            (time, source, lat, lon) float32 264.8 264.9 ... nan

In [18]:
# save
with open(f'{dirpath}/fldas/2017/fldas2017_xarray.pickle', 'wb') as handle:
    pickle.dump(fldas_all, handle, protocol=pickle.HIGHEST_PROTOCOL)